In [2]:
## Import the following libraries.

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import torch
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [17]:
## Read the previously prepared dataset.

data = pd.read_csv("dccc_prepared.csv")
data.head()

,LIMIT_BAL,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,0.010101,0.333333,0.333333,0.051724,0.4,0.4,0.1,0.1,0.0,0.0,...,0.160138,0.080648,0.260979,0.000000,0.000409,0.000000,0.000000,0.000000,0.000000,1
1,0.111111,0.333333,0.666667,0.086207,0.1,0.4,0.2,0.2,0.2,0.4,...,0.163220,0.084074,0.263485,0.000000,0.000594,0.001116,0.001610,0.000000,0.003783,1
2,0.080808,0.333333,0.666667,0.224138,0.2,0.2,0.2,0.2,0.2,0.2,...,0.173637,0.095470,0.272928,0.001738,0.000891,0.001116,0.001610,0.002345,0.009458,0
3,0.040404,0.333333,0.333333,0.275862,0.2,0.2,0.2,0.2,0.2,0.2,...,0.186809,0.109363,0.283685,0.002290,0.001199,0.001339,0.001771,0.002506,0.001892,0
4,0.040404,0.333333,0.333333,0.620690,0.1,0.2,0.1,0.2,0.2,0.2,...,0.179863,0.099633,0.275681,0.002290,0.021779,0.011160,0.014493,0.001615,0.001284,0


In [27]:
## Separate the features from the target.

X = data.iloc[:,:-1]
y = data["default.payment.next.month"]

In [37]:
## Using scikit-learn's train_test_split function, split the dataset into training, validation, and testing sets. Use a 60/20/20% split ratio. Set random_state as 0.

x_new, x_test, y_new, y_test = train_test_split(X,Y, test_size = 0.2, random_state = 0)

x_train, x_dev, y_train, y_dev = train_test_split(x_new, y_new, test_size = 0.2, random_state = 0)

# X_new, X_test, y_new, y_test = train_test_split(X, y,  test_size=0.2, random_state=0)
# dev_per = X_test.shape[0]/X_new.shape[0]
# X_train, X_dev, y_train, y_dev = train_test_split(X_new,  y_new, test_size=dev_per, random_state=0)

In [38]:
print(x_train.shape, y_train.shape)
print(x_dev.shape, y_dev.shape)
print(x_test.shape, y_test.shape)

(19200, 22) (19200,)
(4800, 22) (4800,)
(6000, 22) (6000,)


In [39]:
## Convert the validation and testing sets to tensors, considering that the features matrices should be of type float, while the target matrices should not. Leave the training sets unconverted for the moment as they will undergo further transformations.

# X_train = torch.tensor(x_train.values).float()
# y_train = torch.tensor(y_train.values).float()

x_dev = torch.tensor(x_dev.values).float()
y_dev = torch.tensor(y_dev.values).float()

x_test = torch.tensor(x_test.values).float()
y_test = torch.tensor(y_test.values).float()

In [40]:
y_dev = torch.tensor(y_dev.values)
y_test = torch.tensor(y_test.values)

RuntimeError: Could not infer dtype of builtin_function_or_method

In [41]:
# additional:
print(type(x_train))
print(type(y_train))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [42]:
# additional:
# converting training_set back to a dataframe
# shouldn't have converted it to a tensor in the first place

#x_train = pd.DataFrame(x_train.numpy())
#y_train = pd.DataFrame(y_train.numpy())

In [43]:
## Build a custom module class defining the layers of the network. Include a forward 
## function that specifies the activation functions that will be applied to the output 
## of each layer. Use ReLU for all layers, except for the output, where you should use 
## log_softmax.

class Classifier(nn.Module):  # custom module
    def __init__(self, input_size):
        super().__init__()
        self.hidden_1 = nn.Linear(input_size, 10)
        self.hidden_2 = nn.Linear(10, 10)
        self.hidden_3 = nn.Linear(10, 10)
        self.output = nn.Linear(10, 2)
    def forward(self, x):
        z = F.relu(self.hidden_1(x))
        z = F.relu(self.hidden_2(z))
        z = F.relu(self.hidden_3(z))
        out = F.log_softmax(self.output(z), dim=1)
        return out

In [44]:
## Define all the variables required for the training of the model. Set the number of 
## epochs to 50 and the batch size to 128. Use a learning rate of 0.001.

model = Classifier(x_train.shape[1])
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 50
batch_size = 128
learning_rate = 0.001


In [50]:
## Train the network using the training sets data. Use the validation sets to measure 
## performance. To do so, save the loss and the accuracy for both the training and 
## validation sets in each epoch.

## The training process may take several minutes, depending on your resources. 
## Adding print statements is a good practice to see the progress of the training 
## process.

train_losses, dev_losses, train_acc, dev_acc= [], [], [], []
for e in range(epochs):
    X_, y_ = shuffle(x_train, y_train)
    running_loss = 0
    running_acc = 0
    iterations = 0
    for i in range(0, len(X_), batch_size):
        iterations += 1
        b = i + batch_size
        X_batch = torch.tensor(X_.iloc[i:b,:].values).float()
        y_batch = torch.tensor(y_.iloc[i:b].values).long()
        log_ps = model(X_batch)
        loss = criterion(log_ps, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        ps = torch.exp(log_ps)
        top_p, top_class = ps.topk(1, dim=1)
        running_acc += accuracy_score(y_batch, top_class)
        dev_loss = 0
        acc = 0
        with torch.no_grad():
            log_dev = model(x_dev)
            dev_loss = criterion(log_dev, y_dev)
            ps_dev = torch.exp(log_dev)
            top_p, top_class_dev = ps_dev.topk(1, dim=1)
            acc = accuracy_score(y_dev, top_class_dev)
        train_losses.append(running_loss/iterations)
        dev_losses.append(dev_loss)
        train_acc.append(running_acc/iterations)
        dev_acc.append(acc)
        print("Epoch: {}/{}.. ".format(e+1, epochs),
            "Training Loss: {:.3f}.. ".format(running_loss/iterations),
            "Validation Loss: {:.3f}.. ".format(dev_loss),
            "Training Accuracy: {:.3f}.. ".format(running_acc/ 
                                                  iterations),
            "Validation Accuracy: {:.3f}".format(acc))

RuntimeError: expected scalar type Long but found Float

In [ ]:
## Plot the loss of both sets.

plt.plot(train_losses, label='Training loss')
plt.plot(dev_losses, label='Validation loss')
plt.legend(frameon=False)
plt.show()

In [ ]:
##  Plot the accuracy of both sets.

plt.plot(train_acc, label="Training accuracy")
plt.plot(dev_acc, label="Validation accuracy")
plt.legend(frameon=False)
plt.show()